# Trabalho Final de Pesquisa Operacional - Branch And Bound
**ALUNO**: Guilherme Barboza de Sousa  
**MATRÍCULA**: 20220007418  
**PROFº**: Teobaldo Bulhões  
**APRESENTAÇÃO**: https://youtu.be/WdltUMqmLug

##Introdução

Este trabalho tem como objetivo, realizar a implementação, em Python, do método Branch And Bound para quadro problemas de programação linear inteira binária, problemas que, possuirão todas as funções objetivas como maximização.

##Instalando e Importando Biblioteca MIP


In [1]:
!pip install mip

In [101]:
from mip import *


##Função Para Ler o Arquivo e Extraindo informações

In [197]:
var_coef = [] #coeficientes das variaveis na funcao objetivo
qnt_var = 0 #quantidade de variaveis
qnt_rest = 0 #quantidade de restricoes

def read_file(archive):
  global var_coef, restrictions, restrictionsLimits, num_var, num_rest, qnt_var, qnt_rest
  try:
    with open(archive, 'r') as arq: #abrindo o arquivo

      line = arq.readlines()
      var_res = line[0].split()
      var_coef = line[1].split()

      qnt_var = int(var_res[0]) #quantidade de variaveis
      qnt_rest = int(var_res[1]) #quantidade de restricoes

    restrictions = []  #lista de listas com os coeficientes das restricoes
    restrictionsLimits = [] #lista de limites do modelo

    for i in range(2, len(line)):
      restslimit = line[i].split() #variavel para armazenar os coeficientes e limites das restrições
      restrictions.append(list(map(int, restslimit[:qnt_var])))
      restrictionsLimits.append(int(restslimit[qnt_var]))

    var_coef = list(map(int, var_coef)) #tranformando lista de strings em int

  except FileNotFoundError:
    print(f"O arquivo '{archive}' não foi encontrado.")
  except Exception as e:
    print(f"Erro ao ler o arquivo: {e}")

arc = 'teste4.txt'
read_file(arc)

print(f"Quantidade de Variáveis: {qnt_var}")
print(f"Quantidade de Restrições: {qnt_rest}")
print(f"Coeficientes Variaveis Função Objetivo: {var_coef}\n")
print("Coeficientes das Restrições: ")
for i in restrictions:
  print(i)
print(f"\nLimites das Restrições: {restrictionsLimits}")

Quantidade de Variáveis: 9
Quantidade de Restrições: 9
Coeficientes Variaveis Função Objetivo: [9, 7, 10, 7, 9, 6, 8, 4, 9]

Coeficientes das Restrições: 
[4, 9, 4, 1, 9, 6, 3, 6, 1]
[3, 7, 8, 7, 6, 3, 5, 9, 4]
[9, 3, 6, 5, 7, 1, 1, 3, 9]
[5, 9, 6, 5, 9, 7, 8, 7, 8]
[7, 7, 4, 1, 3, 4, 8, 1, 9]
[1, 6, 6, 1, 6, 7, 3, 8, 7]
[6, 6, 8, 6, 10, 8, 1, 4, 4]
[9, 1, 9, 7, 10, 5, 6, 2, 5]
[2, 7, 6, 5, 1, 1, 9, 2, 1]

Limites das Restrições: [40, 80, 40, 10, 10, 10, 70, 10, 20]


##Preenchimento do Modelo Relaxado


In [198]:


rmodel = Model()

# adicionando a quantidade de variaveis
x = [rmodel.add_var(var_type=CONTINUOUS, lb=0, ub=1) for i in range(qnt_var)]
#var_type = CONTINUOUS, para admitir valores decimais
#lb=0, limite inferior 0
#ub=1, limite superior 1

# formando a função objetivo multiplicando os indices de var_coef com as variaveis
rmodel.objective = maximize(xsum(var_coef[i] * x[i] for i in range(qnt_var)))

# adicionando as restrições ao modelo
for i in range(qnt_rest):
    rests = xsum(restrictions[i][j] * x[j] for j in range(qnt_var))
    rmodel.add_constr(rests <= restrictionsLimits[i])

##Função Para Resolver o Modelo

In [199]:
def solve(model):
  model.optimize()
  if model.num_solutions:
    print("\n")
    print(model.status)
    print(f"Z  = {model.objective_value:.2f}")

    print('Solução:')
    for i, v in enumerate(model.vars):
      var_name = f"X{i+1}"
      var_value = v.x
      print(f"{var_name} = {var_value:.2f}")

print("Modelo Relaxado")
solve(rmodel)
print("--------------------------------------")

Modelo Relaxado


OptimizationStatus.OPTIMAL
Z  = 13.57
Solução:
X1 = 0.00
X2 = 0.00
X3 = 0.71
X4 = 0.00
X5 = 0.00
X6 = 0.00
X7 = 0.00
X8 = 0.00
X9 = 0.71
--------------------------------------


## Listar os Valores das Váriaveis do Modelo Relaxado Otimizado

In [201]:
def list_vars(m):  #m = model
  solution_values = [] #lista para armazenar os valores das variaveis de m
  for i, v in enumerate(m.vars):
          var_value = v.x
          solution_values.append(var_value)
          solution_values = list(map(float, solution_values))

  return solution_values

list_vars(rmodel)

[0.0, 0.0, 0.7142857142857143, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7142857142857144]

##Função para Avaliar as Variáveis

In [202]:
def vars_int(modelvar): #modelvar = variaveis de um modelo
    for index, value in enumerate(modelvar):
        if not isinstance(value, int) and not value.is_integer():
            return index
    return True

vars_int(list_vars(rmodel))

2

## Função de Ramificação

In [203]:
def bnb(m):
  li=float('-inf') #li = limite inferior
  ls = rmodel.objective_value #ls = limite superior
  node = [m]
  count = 0 #contagem pilha

  while node:

    solve(node[-1])
    if node[-1].status == OptimizationStatus.OPTIMAL and node[-1].objective_value >= li and node[-1].objective_value <= ls:
      if vars_int(list_vars(node[-1])) is not True:
        change = vars_int(list_vars(node[-1])) #indice para mudar

        node.append(node[-1].copy())
        count+=1
        node[-1] += x[change] == 0
        continue

      li = node[-1].objective_value
      for i in range(count):
        del node[-1]
      count = 0

      if vars_int(list_vars(node[-1])) is not True:
        change = vars_int(list_vars(node[-1])) #indice para mudar
        node.append(node[-1].copy())
        node[-1] += x[change] == 1
        del node[0]
        continue

    for i in range(count):
      del node[-1]
    count = 0
    break
  print('--------------------\n')
  return f"Solução Ótima: {li}"


bnb(rmodel)



OptimizationStatus.OPTIMAL
Z  = 13.57
Solução:
X1 = 0.00
X2 = 0.00
X3 = 0.71
X4 = 0.00
X5 = 0.00
X6 = 0.00
X7 = 0.00
X8 = 0.00
X9 = 0.71


OptimizationStatus.OPTIMAL
Z  = 13.08
Solução:
X1 = 0.38
X2 = 0.00
X3 = 0.00
X4 = 0.38
X5 = 0.00
X6 = 0.00
X7 = 0.00
X8 = 0.00
X9 = 0.77


OptimizationStatus.OPTIMAL
Z  = 12.58
Solução:
X1 = 0.00
X2 = 0.00
X3 = 0.00
X4 = 0.97
X5 = 0.00
X6 = 0.00
X7 = 0.00
X8 = 0.00
X9 = 0.65


OptimizationStatus.OPTIMAL
Z  = 11.00
Solução:
X1 = 0.00
X2 = 0.00
X3 = 0.00
X4 = 0.00
X5 = 0.22
X6 = 0.00
X7 = 0.00
X8 = 0.00
X9 = 1.00


OptimizationStatus.OPTIMAL
Z  = 10.65
Solução:
X1 = 0.00
X2 = 0.00
X3 = 0.00
X4 = 0.00
X5 = 0.00
X6 = 0.32
X7 = 0.00
X8 = 0.00
X9 = 0.97


OptimizationStatus.OPTIMAL
Z  = 10.50
Solução:
X1 = 0.00
X2 = 0.00
X3 = 0.00
X4 = 0.00
X5 = 0.00
X6 = 0.00
X7 = 0.10
X8 = 0.17
X9 = 1.00


OptimizationStatus.OPTIMAL
Z  = 10.38
Solução:
X1 = 0.00
X2 = 0.13
X3 = 0.00
X4 = 0.00
X5 = 0.00
X6 = 0.00
X7 = 0.00
X8 = 0.12
X9 = 1.00


OptimizationStatus.OPTIMA

'Solução Ótima: 10.0'